In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

In [ ]:
size = 10 ** 6
start_time = pd.to_datetime('20140824 7:0:0')
end_time = pd.to_datetime('20140824 8:0:0')
# start_time = '20140824 7'
# end_time = '20140824 8'
hour_dataframe = pd.DataFrame()
for chunk in pd.read_csv('20140824_train.txt', chunksize=size):
    chunk.columns = ['taxi_id', 'latitude', 'longitude', 'passenger', 'time']
    chunk.time = chunk.time.apply(pd.to_datetime)
    mask = (chunk.time >= start_time) & (chunk.time < end_time)
    hour_dataframe = pd.concat([hour_dataframe, chunk.loc[mask]])

分块提取特定时间的出租车客流数据

In [ ]:
up_off_pairs = []
grouped = hour_dataframe.groupby('taxi_id', sort=False)
for taxi_id, group in grouped:
    selected_group_length = len(group)
    group = group.sort_values(by=['time'])
    i = 0
    while i < selected_group_length - 1:
        if ((group.iloc[[i]].passenger == 1).bool() & (group.iloc[[i + 1]].passenger != 0).bool()):
            up_point_row = group.iloc[[i]]
            up_off_point = [[up_point_row.longitude, up_point_row.latitude]]
            j = i + 1
            while j < selected_group_length:
                print('current_j is %s' % j)
                if (group.iloc[[j]].passenger == 0).bool():
                    off_point_row = group.iloc[[j - 1]]
                    up_off_point.append([off_point_row.longitude, off_point_row.latitude])
                    i = j + 1
                    break
                else:
                    if j == selected_group_length - 1:
                        up_off_point.append([group.iloc[[j]].longitude, group.iloc[[j]].latitude])
                        i = j
                        break
                    else:
                        j += 1
            if len(up_off_point) == 2:
                up_off_pairs.append(up_off_point)
        else:
            i += 1
print(up_off_pairs)

对每辆出租车在特定时间内的上客、下客点进行统计。